# Assignment 1

In [6]:
from collections import defaultdict, deque
import itertools
import numpy as np

import time

In [7]:
class Graph:
    def __init__(self, U_size, V_size):
        self.U = list(range(1, U_size + 1))  # Nodes in set U
        self.V = list(range(U_size + 1, U_size + V_size + 1))  # Nodes in fixed layer U
        self.edges = []  # List of edges (u, v, weight)
        self.constraints = defaultdict(list)  # Constraints as adjacency list for V
        self.in_degree = {v: 0 for v in self.V}  # Dictionary to store in-degree of nodes in V

    def add_node_U(self, node):
        self.U.append(node)

    def add_node_V(self, node):
        self.V.append(node)
        self.in_degree[node] = 0  # Initialize in-degree for nodes in V

    def add_edge(self, u, v, weight):
        self.edges.append((u, v, weight))

    def add_constraint(self, v1, v2):
        self.constraints[v1].append(v2)
        self.in_degree[v2] += 1  # Update in-degree due to precedence constraint

In [8]:
def load_instance(filename):
    with open(filename, 'r') as file:
        U_size, V_size, C_size, E_size = map(int, file.readline().split())
        graph = Graph(U_size, V_size)

        # Read constraints section
        line = file.readline().strip()
        while line != "#constraints":
            line = file.readline().strip()

        for _ in range(C_size):
            v, v_prime = map(int, file.readline().split())
            graph.add_constraint(v, v_prime)

        # Read edges section
        line = file.readline().strip()
        while line != "#edges":
            line = file.readline().strip()

        for _ in range(E_size):
            u, v, weight = file.readline().split()
            graph.add_edge(int(u), int(v), float(weight))

    return graph

In [9]:
def cost_function(graph, permutation):
    # Create a dictionary for quick lookup of node positions in the current ordering
    position = {node: idx for idx, node in enumerate(permutation)}
    total_cost = 0
    # Iterate over all pairs of edges to count crossings
    for (u1, v1, w1), (u2, v2, w2) in itertools.combinations(graph.edges, 2):
        # Check if edges cross based on their positions
        if (u1 < u2 and position[v1] > position[v2]) or (u1 > u2 and position[v1] < position[v2]):
            # Add the sum of weights for the crossing edges to the total cost
            total_cost += w1 + w2

    return total_cost

## Question 1: Think about a meaningful real-world application for this problem and briefly describe it

Application: Railway Scheduling and Track Layout

- Nodes in $U$ could represent fixed train stations along a route, where trains must stop in a specific order
- Nodes in $V$ could reoresent trains/train routes to be scheduled within the network
- Weighetd edges would inidcate the relationship between trains/train routes and tracks, with higher weights representing either higher traffic, longer distance, lower importance in terms of scheduling (in case of for exaple local vs. high speed train)
- constraints in C would enforce rules like the fact that certain trains need to arrive before others or that trains using the same tracks do not overlap.

The objective then would be to arrange the schedules in V to minimize the crossings of tracks while satisfying all contraints in C. This would lead to improved security and efficiency as minimzing track crossings would reduce the likelihood of collisions or delaysdue to track conflicts.

## Question 2: Develop a meaningful deterministic construction heuristic

A meaningful determinitic contruction heuristic could be one based on a greedy approach:
- Inititalize an empty ordered permuation list $\pi$ for nodes in $V$
- Compute for each node in $V$ the number of constraints that require other nodes to come before it
- Create a list of candidates with nodes in $V$ that have no predecessors

- For every node in the candidates and until $V$ is empty
  - calculate the total weight of edges connecting it to U
  - select the node with the lowest total edge weight to nodes in $U$ and append it to the final permutation list $\pi$

  - update the in-degree of nodes that had this as a constraint reducing it by 1

  - if the in-degree of any node in V reaches zero add it to the candidates




In [10]:
class DeterministicConstruction:
    def __init__(self, graph):
        self.graph = graph
        self.pi = []  # store the final order of nodes in V

    def greedy_construction(self):
        # Initialize candidates with in-degree 0 nodes
        candidates = deque([v for v in self.graph.V if self.graph.in_degree[v] == 0])

        while candidates:
            # Select node with lowest edge weight to nodes in U
            best_node = None
            best_weight_sum = float('inf')  # Minimize total weight

            for v in candidates:
                # Calculate the sum of edge weights for the current candidate node
                weight_sum = sum(weight for u, v2, weight in self.graph.edges if v2 == v)
                if weight_sum < best_weight_sum:
                    best_weight_sum = weight_sum
                    best_node = v

            # Add the selected best_node to the pi
            self.pi.append(best_node)
            candidates.remove(best_node)

            # Update in-degrees based on the constraints, and add new candidates
            for v_next in self.graph.constraints[best_node]:
                self.graph.in_degree[v_next] -= 1
                if self.graph.in_degree[v_next] == 0:
                    candidates.append(v_next)

        return self.pi

    def calculate_cost(self):
        # Create a dictionary for quick lookup of node positions in the current ordering
        position = {node: idx for idx, node in enumerate(self.pi)}
        total_cost = 0

        # Iterate over all pairs of edges to count crossings
        for (u1, v1, w1), (u2, v2, w2) in itertools.combinations(self.graph.edges, 2):
            # Check if edges cross based on their positions
            if (u1 < u2 and position[v1] > position[v2]) or (u1 > u2 and position[v1] < position[v2]):
                # Add the sum of weights for the crossing edges to the total cost
                total_cost += w1 + w2

        return total_cost


In [11]:
# Example usage with a small problem instance
graph = Graph(5, 5)

# Adding edges with weights
graph.add_edge(1, 7, 1)
graph.add_edge(2, 8, 2)
graph.add_edge(3, 9, 1)
graph.add_edge(4, 10, 3)
graph.add_edge(5, 6, 11)

# Adding precedence constraints
graph.add_constraint(7, 10)
graph.add_constraint(8, 9)

# Create an MWCCP solution instance and use the greedy heuristic
solution = DeterministicConstruction(graph)
ordering = solution.greedy_construction()

print("The initial ordering of nodes in V:", ordering)

print(solution.calculate_cost())

The initial ordering of nodes in V: [7, 8, 9, 10, 6]
0


In [12]:
permutation = np.random.permutation(graph.V)
print(permutation)

obj_cost = cost_function(graph, permutation)
print(obj_cost)

obj_cost = cost_function(graph, ordering)
print(obj_cost)

[ 9 10  7  8  6]
14
0


In [13]:
graph_loaded = load_instance('in.txt')

solution = DeterministicConstruction(graph_loaded)
ordering = solution.greedy_construction()

print("The initial ordering of nodes in V:", ordering)

print(solution.calculate_cost())


The initial ordering of nodes in V: [35, 43, 39, 48, 27, 49, 26, 31, 30, 34, 37, 38, 45, 46, 28, 40, 33, 41, 32, 42, 36, 47, 50, 29, 44]
94914.0


## Question 3: Derive a randomized construction heuristic to be applied iteratively

The randomized heuristic could follow the same process as the deterministic one, but then, when choosing the node in $V$ to add to the final permutation list $\pi$, instead of always choosing the one that has a minimum weighted sum of edges to $U$, pick one randomly, with a probability inversely propotional to this sum.

In [ ]:
class RandomizedConstruction:
    def __init__(self, graph):
        self.graph = graph
        self.pi = []  # This will store the final order of nodes in V

    def greedy_construction(self):
        # Initialize candidates with in-degree 0 nodes
        candidates = deque([v for v in self.graph.V if self.graph.in_degree[v] == 0])

        while candidates:
            # Select node with lowest edge weight to nodes in U
            best_node = None

            sums = []
            for v in candidates:
                # Calculate the sum of edge weights for the current candidate node
                weight_sum = sum(weight for u, v2, weight in self.graph.edges if v2 == v)
                sums.append(weight_sum)

            probs = sums / np.sum(sums)
            probs = 1 / probs
            probs = np.exp(probs) / np.sum(np.exp(probs), axis=0)

            best_node = np.random.choice(candidates, p=probs)

            # Add the selected best_node to the pi
            self.pi.append(best_node)
            candidates.remove(best_node)
            # Update in-degrees based on the constraints, and add new candidates
            for v_next in self.graph.constraints[best_node]:
                self.graph.in_degree[v_next] -= 1
                if self.graph.in_degree[v_next] == 0:
                    candidates.append(v_next)

        return self.pi

    def calculate_cost(self):
        # Create a dictionary for quick lookup of node positions in the current ordering
        position = {node: idx for idx, node in enumerate(self.pi)}
        total_cost = 0

        # Iterate over all pairs of edges to count crossings
        for (u1, v1, w1), (u2, v2, w2) in itertools.combinations(self.graph.edges, 2):
            # Check if edges cross based on their positions
            if (u1 < u2 and position[v1] > position[v2]) or (u1 > u2 and position[v1] < position[v2]):
                # Add the sum of weights for the crossing edges to the total cost
                total_cost += w1 + w2

        return total_cost
    
class RandomizedConstruction_v2:
    def __init__(self, graph, alpha):
        self.graph = graph
        self.alpha = alpha
        self.pi = []

    def randomized_construction(self):
        # Initialize candidates with in-degree 0 nodes
        candidates = deque([v for v in self.graph.V if self.graph.in_degree[v] == 0])

        while candidates:
            # Calculate cost for each candidate by temporarily appending it to the current solution
            candidates_costs = []
            for el in candidates:
                self.pi.append(el)  # Temporarily add to the solution
                cost = self.calculate_cost()  # Calculate the cost with this candidate included
                candidates_costs.append((el, cost))
                self.pi.pop()  # Remove to restore original solution for next candidate evaluation

            # Sort candidates by cost in ascending order
            candidates_costs.sort(key=lambda x: x[1])
            cmin = candidates_costs[0][1]
            cmax = candidates_costs[-1][1]
            thresh = cmin + self.alpha * (cmax - cmin)

            # Create the restricted candidate list (RCL) based on threshold
            rcl = [el for el, cost in candidates_costs if cost <= thresh]
            if not rcl:
                break

            # Randomly select from the RCL
            best_node = np.random.choice(rcl)
            self.pi.append(best_node)
            candidates.remove(best_node)

            # Update in-degrees based on the constraints, adding new candidates as in-degrees drop to 0
            for v_next in self.graph.constraints[best_node]:
                self.graph.in_degree[v_next] -= 1
                if self.graph.in_degree[v_next] == 0:
                    candidates.append(v_next)
        if len(self.pi) == len(self.graph.V):
            return self.pi
        else:
            return self.pi + [v for v in self.graph.V if v not in self.pi]


    def calculate_cost(self):
        # Create a dictionary for quick lookup of node positions in the current ordering
        position = {node: idx for idx, node in enumerate(self.pi)}
        total_cost = 0

        # Iterate over all pairs of edges to count crossings
        for (u1, v1, w1), (u2, v2, w2) in itertools.combinations(self.graph.edges, 2):
            # Only calculate crossings if both nodes of each edge are in `self.pi`
            if v1 in position and v2 in position:
                # Check if edges cross based on their positions
                if (u1 < u2 and position[v1] > position[v2]) or (u1 > u2 and position[v1] < position[v2]):
                    # Add the sum of weights for the crossing edges to the total cost
                    total_cost += w1 + w2

        return total_cost



In [15]:
graph = Graph(5, 5)

# Adding edges with weights
graph.add_edge(1, 7, 1)
graph.add_edge(2, 8, 2)
graph.add_edge(3, 9, 1)
graph.add_edge(4, 10, 3)
graph.add_edge(5, 6, 11)

# Adding precedence constraints
graph.add_constraint(7, 10)
graph.add_constraint(8, 9)

solution = RandomizedConstruction(graph)
ordering = solution.greedy_construction()

print("The initial ordering of nodes in V:", ordering)

print(solution.calculate_cost())

The initial ordering of nodes in V: [7, 10, 8, 9, 6]
9


In [16]:
graph_loaded = load_instance('in.txt')

solution = RandomizedConstruction(graph_loaded)
ordering = solution.greedy_construction()

print("The initial ordering of nodes in V:", ordering)

print(solution.calculate_cost())


The initial ordering of nodes in V: [48, 35, 43, 39, 49, 31, 27, 26, 30, 38, 34, 46, 37, 45, 42, 28, 41, 36, 50, 47, 33, 40, 44, 29, 32]
96210.0


## 4. Develop or make use of a framework for basic local search which is able to deal with: different neighborhood structures and different step functions (first-improvement, best-improvement, random)


In [17]:
import random


class LocalSearch:
    def __init__(self, initial_solution, neighborhood_function, step_function, objective_function, max_iter=500):
        """
        Local Search framework.

        Args:
        - initial_solution: Starting solution for the search
        - neighborhood_function: Function to generate neighbor solutions
        - step_function: Function to select next step in search
        - objective_function: Function to compute the cost of a solution
        - max_iter: Maximum number of iterations for the search
        """
        self.current_solution = initial_solution
        self.neighborhood_function = neighborhood_function
        self.step_function = step_function
        self.objective_function = objective_function
        self.max_iter = max_iter

    def local_search(self):
        best_solution = self.current_solution
        best_cost = self.objective_function(best_solution)

        for i in range(self.max_iter):

            neighbors = self.neighborhood_function(self.current_solution)
            next_solution = self.step_function(neighbors, self.objective_function)
            next_cost = self.objective_function(next_solution)

            if next_cost < best_cost:
                best_solution = next_solution
                best_cost = next_cost
                continue

            if best_cost == 0:
                break

            self.current_solution = next_solution

        print("Required iterations:", i)
        return best_solution, best_cost


def best_improvement(neighbors, objective_function):
    if not neighbors:
        return None

    best_neighbor = None
    best_cost = float('inf')

    for neighbor in neighbors:
        cost = objective_function(neighbor)
        if cost < best_cost:
            best_neighbor = neighbor
            best_cost = cost

    return best_neighbor


def first_improvement(neighbors, objective_function):
    for neighbor in neighbors:
        if objective_function(neighbor) < objective_function(neighbors[0]):
            return neighbor
    return neighbors[0]


def random_neighbor(neighbors, objective_function):
    return random.choice(neighbors)


def cost_function(graph, permutation):  # same function as above
    if permutation is None:
        return float('inf')
    # Create a dictionary for quick lookup of node positions in the current ordering
    position = {node: idx for idx, node in enumerate(permutation)}
    total_cost = 0
    # Iterate over all pairs of edges to count crossings
    for (u1, v1, w1), (u2, v2, w2) in itertools.combinations(graph.edges, 2):
        # Check if edges cross based on their positions
        if (u1 < u2 and position[v1] > position[v2]) or (u1 > u2 and position[v1] < position[v2]):
            # Add the sum of weights for the crossing edges to the total cost
            total_cost += w1 + w2

    return total_cost

## 5. Develop at least three different meaningful neighborhood structures

In [18]:
def is_valid_operator(solution, constraints):
    position = {node: idx for idx, node in enumerate(solution)}
    for first, second in constraints:
        if position[second] < position[first]:
            return False
    return True

### 1. Swap operator

In [19]:
def swap_neighborhood(solution, constraints):
    if solution is None:
        return []

    neighbors = []
    if not isinstance(solution, list):
        solution = list(solution)

    for i in range(len(solution) - 1):
        neighbor = solution.copy()
        neighbor[i], neighbor[i + 1] = neighbor[i + 1], neighbor[i]
        if is_valid_operator(neighbor, constraints) and neighbor not in neighbors:
            neighbors.append(neighbor)

    return neighbors

In [20]:
#initial_solution = np.random.permutation(graph.V)
# initial_solution = ordering
initial_solution = [6, 8, 9, 7, 10]

print("The initial ordering of nodes in V:", initial_solution)
print(cost_function(graph, initial_solution))

constraints = [(key, item[0]) for key, item in graph.constraints.items() if len(item) > 0]

neighborhood_function = lambda sol: swap_neighborhood(sol, constraints)
objective_function = lambda sol: cost_function(graph, sol)

# Step functions
for step_fun in [best_improvement, first_improvement, random_neighbor]:
    print("----")
    local_search = LocalSearch(initial_solution, neighborhood_function, step_fun, objective_function)
    best_solution, best_cost = local_search.local_search()

    print(step_fun.__name__)
    print("Improvement:", best_solution)
    print(best_cost)

The initial ordering of nodes in V: [6, 8, 9, 7, 10]
56
----
Required iterations: 11
best_improvement
Improvement: [7, 8, 9, 10, 6]
0
----
Required iterations: 11
first_improvement
Improvement: [7, 8, 9, 10, 6]
0
----
Required iterations: 58
random_neighbor
Improvement: [7, 8, 9, 10, 6]
0


### 2. Insert operator

In [21]:
def insert_neighborhood(solution, constraints):
    if solution is None:
        return []

    neighbors = set()
    solution = list(solution)

    for i in range(len(solution)):
        for j in range(len(solution)):
            if i != j:
                neighbor = solution.copy()
                neighbor.insert(j, neighbor.pop(i))
                neighbor = tuple(neighbor)
                if is_valid_operator(neighbor, constraints) and neighbor not in neighbors:
                    neighbors.add(neighbor)

    return list(neighbors)

In [22]:
#initial_solution = ordering
#initial_solution = np.random.permutation(graph.V)

initial_solution = [6, 8, 7, 10, 9]
print("The initial ordering of nodes in V:", initial_solution)
print(cost_function(graph, initial_solution))

neighborhood_function = lambda sol: insert_neighborhood(sol, constraints)
objective_function = lambda sol: cost_function(graph, sol)

# Improvement step function
for step_fun in [best_improvement, first_improvement, random_neighbor]:
    print("----")
    local_search = LocalSearch(initial_solution, neighborhood_function, step_fun, objective_function)
    best_solution, best_cost = local_search.local_search()

    print(step_fun.__name__)
    print("Improvement:", best_solution)
    print(best_cost)


The initial ordering of nodes in V: [6, 8, 7, 10, 9]
58
----
Required iterations: 5
best_improvement
Improvement: (7, 8, 9, 10, 6)
0
----
Required iterations: 5
first_improvement
Improvement: (7, 8, 9, 10, 6)
0
----
Required iterations: 42
random_neighbor
Improvement: (7, 8, 9, 10, 6)
0


### 3. two-opt operator

In [23]:
def two_opt_neighborhood(solution, constraints):
    if solution is None:
        return []

    neighbors = set()

    for i in range(len(solution)):
        neighbor_first = tuple(solution[:i])
        for j in range(i + 1, len(solution)):
            neighbor = neighbor_first + tuple(reversed(solution[i: j + 1])) + tuple(solution[j + 1:])
            # neighbor[i: j+1] = list(reversed(neighbor[i:j+1]))
            if is_valid_operator(neighbor, constraints) and neighbor not in neighbors:
                neighbors.add(neighbor)
    return list(neighbors)

In [24]:
#initial_solution = ordering

initial_solution = [6, 8, 7, 10, 9]

print("The initial ordering of nodes in V:", initial_solution)
print(cost_function(graph, initial_solution))

neighborhood_function = lambda sol: two_opt_neighborhood(sol, constraints)
objective_function = lambda sol: cost_function(graph, sol)

for step_fun in [best_improvement, first_improvement, random_neighbor]:
    print("----")
    local_search = LocalSearch(initial_solution, neighborhood_function, step_fun, objective_function)
    best_solution, best_cost = local_search.local_search()

    print(step_fun.__name__)
    print("Improvement:", best_solution)
    print(best_cost)


The initial ordering of nodes in V: [6, 8, 7, 10, 9]
58
----
Required iterations: 3
best_improvement
Improvement: (7, 8, 9, 10, 6)
0
----
Required iterations: 3
first_improvement
Improvement: (7, 8, 9, 10, 6)
0
----
Required iterations: 80
random_neighbor
Improvement: (7, 8, 9, 10, 6)
0


## 6. Develop or make use of a Variable Neighborhood Descent (VND) framework which uses your neighborhood structures.

In [25]:
class VND:
    def __init__(self, initial_solution, neighborhood_structures, constraints, objective_function, step_function,
                 max_iter=500, verbose=True):
        """
        Variable Neighborhood Descent framework.

        Args:
        - initial_solution: Starting solution for the search
        - neighborhood_structures: List of neighborhood functions to use
        - constraints: List of constraints for the problem
        - objective_function: Function to compute the cost of a solution
        - step_function: Function to select next step in search
        - max_iter: Maximum number of iterations for the search
        """
        self.current_solution = initial_solution
        self.neighborhood_structures = neighborhood_structures
        self.objective_function = objective_function
        self.step_function = step_function
        self.max_iter = max_iter
        self.constraints = constraints
        self.verbose = verbose

    def vnd(self, solution=None):
        if solution is not None:
            self.current_solution = solution

        best_solution = self.current_solution
        best_cost = self.objective_function(best_solution)
        l = 0

        for i in range(self.max_iter):

            # select neighborhood structure l
            neighborhood_function = self.neighborhood_structures[l]

            neighbors = neighborhood_function(self.current_solution, self.constraints)
            next_solution = self.step_function(neighbors, self.objective_function)  # find local optimum in neighborhood

            if next_solution is not None:
                next_cost = self.objective_function(next_solution)

                if next_cost < best_cost:
                    self.current_solution = best_solution
                    best_solution = next_solution
                    best_cost = next_cost
                    l = 0
                    continue
                else:
                    l += 1

            if l >= len(self.neighborhood_structures):
                break

            if best_cost == 0:
                break
        
        if self.verbose:
            print("Required iterations for VND:", i + 1)
        return best_solution, best_cost

In [26]:
neighborhood_structures = [swap_neighborhood, insert_neighborhood, two_opt_neighborhood]

# initial_solution = [6, 7, 8, 9, 10]
initial_solution = ordering
graph = graph_loaded
constraints = [(key, item[0]) for key, item in graph.constraints.items() if len(item) > 0]

print("The initial ordering of nodes in V:", initial_solution)
print(cost_function(graph, initial_solution))

objective_function = lambda sol: cost_function(graph, sol)

variable_neigh_descent = VND(initial_solution, neighborhood_structures, constraints, objective_function,
                             best_improvement)

best_solution, best_cost = variable_neigh_descent.vnd()

print("Improvement:", best_solution)
print(best_cost)

The initial ordering of nodes in V: [48, 35, 43, 39, 49, 31, 27, 26, 30, 38, 34, 46, 37, 45, 42, 28, 41, 36, 50, 47, 33, 40, 44, 29, 32]
96210.0
Required iterations for VND: 6
Improvement: (33, 48, 35, 43, 39, 49, 31, 27, 26, 30, 38, 34, 46, 37, 45, 42, 28, 41, 36, 50, 47, 40, 44, 29, 32)
92663.0


## 7. Implement a Greedy Randomized Adaptive Search Procedure (GRASP) using your randomized construction heuristic and an effective neighborhood structure with one step function or (a variant of) your VND. Note that the union of existing neighborhood structures also constitutes a (composite) neighborhood structure.

In [ ]:
class GRASP:
    def __init__(self, graph, alpha, max_iterations, neighborhood_structures, objective_function, step_function, constraints, max_iter_vnd=100):
        """
        GRASP framework with randomized construction and local search.

        Args:
        - graph: The input graph structure.
        - alpha: Parameter for controlling greediness in construction phase (0 = purely greedy, 1 = purely random).
        - max_iterations: Maximum number of GRASP iterations.
        - neighborhood_structures: List of neighborhood functions for local search.
        - objective_function: Function to compute the cost of a solution.
        - step_function: Function for selecting the best solution in a neighborhood.
        """
        self.graph = graph
        self.alpha = alpha
        self.max_iterations = max_iterations
        self.neighborhood_structures = neighborhood_structures
        self.objective_function = objective_function
        self.step_function = step_function
        self.constraints = constraints
        self.max_iter_vnd = max_iter_vnd

    def run(self):
        best_solution = None
        best_cost = float('inf')

        for iteration in range(self.max_iterations):
            # Construction Phase: Generate an initial solution
            constructor = RandomizedConstruction_v2(self.graph, self.alpha)
            initial_solution = constructor.randomized_construction()
            while len(initial_solution) != len(self.graph.V):
                initial_solution = constructor.randomized_construction()

            # Local Search Phase: Refine the initial solution
            local_search_constructor = VND(initial_solution, self.neighborhood_structures, self.constraints, self.objective_function, self.step_function, self.max_iter_vnd)
            local_search_solution, local_search_cost = local_search_constructor.vnd()


            # Update the best solution if the improved solution is better
            if local_search_cost < best_cost:
                best_solution = local_search_solution
                best_cost = local_search_cost

            print(f"Iteration {iteration + 1}: Cost = {local_search_cost}, Best Cost = {best_cost}")

            if best_cost == 0:
                break

        return best_solution, best_cost


In [ ]:
graph = load_instance('in.txt')
neighborhood_structures = [swap_neighborhood, insert_neighborhood, two_opt_neighborhood]
constraints = [(key, item[0]) for key, item in graph.constraints.items() if len(item)>0]
objective_function = lambda sol: cost_function(graph, sol)
step_function = first_improvement

grasp = GRASP(graph, 0.5, 50, neighborhood_structures, objective_function, step_function, constraints)
best_solution, best_cost = grasp.run()

print("Improvement:", best_solution)
print(best_cost)

## 8. Implement one of the following metaheuristics:
### • General Variable Neighborhood Search (GVNS) on top of your VND
### • Simulated Annealing (SA)
### • Tabu Search (TS)

In [34]:
def swap_neighborhood_shake(solution, constraints):
    if solution is None:
        return []

    # Randomly select two nodes to swap
    while True:
        i, j = random.sample(range(len(solution)), 2)
        neighbor = solution.copy() if isinstance(solution, list) else list(solution)
        neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
        if is_valid_operator(neighbor, constraints):
            return neighbor
        
def n_swap_neighborhood_shake_generator(n=2):
    def shake(solution, constraints):
        if solution is None:
            return []
        
        neighbor = solution.copy() if isinstance(solution, list) else list(solution)
        for _ in range(n):
            neighbor = swap_neighborhood_shake(neighbor, constraints)
        return neighbor
    
    return shake    
        
def insert_neighborhood_shake(solution, constraints):
    if solution is None:
        return []

    # Randomly select two nodes to swap
    while True:
        i, j = random.sample(range(len(solution)), 2)
        neighbor = solution.copy() if isinstance(solution, list) else list(solution)
        neighbor.insert(j, neighbor.pop(i))
        if is_valid_operator(neighbor, constraints):
            return neighbor
        
        
def two_opt_neighborhood_shake(solution, constraints):
    if solution is None:
        return []

    # Randomly select two nodes to swap
    while True:
        i, j = random.sample(range(len(solution)), 2)
        neighbor_first = tuple(solution[:i])
        neighbor = neighbor_first + tuple(reversed(solution[i: j + 1])) + tuple(solution[j + 1:])
        if is_valid_operator(neighbor, constraints):
            return neighbor

In [43]:
class GVNS:
    def __init__(self, initial_solution, shaking_neighbourhood, local_search_neighbourhoods, constraints,
                 objective_function, max_iter=500, verbose=True):
        """
        General Variable Neighborhood Search framework.

        Args:
        - initial_solution: Starting solution for the search
        - shaking_neighbourhood: Neighborhood function for shaking
        - local_search_neighbourhoods: List of neighborhood functions for local search
        - objective_function: Function to compute the cost of a solution
        - max_iter: Maximum number of iterations for the search
        """
        self.current_solution = initial_solution
        self.shaking_neighbourhood = shaking_neighbourhood
        self.local_search_neighbourhoods = local_search_neighbourhoods
        self.objective_function = objective_function
        self.max_iter = max_iter
        self.constraints = constraints
        self.VND = VND(initial_solution, local_search_neighbourhoods, constraints, objective_function, best_improvement, max_iter, False)
        self.verbose = verbose

    def find_solution(self):
        self.current_solution, best_cost = self.VND.vnd()
        best_solution = self.current_solution

        for i in range(self.max_iter):
            k = 0
            while k < len(self.shaking_neighbourhood):
                # pick random solution from the shaking neighbourhood
                shaken_x = self.shaking_neighbourhood[k](best_solution, self.constraints)
                new_solution, new_cost = self.VND.vnd(shaken_x)

                if new_cost < best_cost:
                    best_solution, best_cost = new_solution, new_cost
                    k = 0
                else:
                    k += 1

            if best_cost == 0:
                break
        if self.verbose:
            print("Required iterations for GVNS:", i + 1)
        return best_solution, best_cost

In [42]:
shaking_neighbourhood = [swap_neighborhood_shake, n_swap_neighborhood_shake_generator(2), n_swap_neighborhood_shake_generator(3), insert_neighborhood_shake, two_opt_neighborhood_shake]
local_search_neighbourhoods = [swap_neighborhood, insert_neighborhood, two_opt_neighborhood]

# initial_solution = ordering
# initial_solution = [6, 8, 7, 10, 9]
graph = graph_loaded
initial_solution = ordering
constraints = [(key, item[0]) for key, item in graph.constraints.items() if len(item) > 0]

print("The initial ordering of nodes in V:", initial_solution)
print(cost_function(graph, initial_solution))

objective_function = lambda sol: cost_function(graph, sol)

gvns = GVNS(initial_solution, shaking_neighbourhood, local_search_neighbourhoods, constraints, objective_function, max_iter=50)

best_solution, best_cost = gvns.find_solution()

print("Improvement:", best_solution)
print(best_cost)


The initial ordering of nodes in V: [48, 35, 43, 39, 49, 31, 27, 26, 30, 38, 34, 46, 37, 45, 42, 28, 41, 36, 50, 47, 33, 40, 44, 29, 32]
96210.0
Required iterations for GVNS: 50
Improvement: [35, 33, 31, 38, 40, 37, 41, 44, 49, 28, 29, 37, 41, 44, 49, 28, 29, 45, 46, 47, 32, 50, 42, 39, 48, 26, 30, 27, 43, 34, 36]
80230.0
